In [ ]:
import numpy as np
import pandas as pd
import scipy

from bin.utils.read_datasets import read_sipgate_dataset
from bin.utils.clean_sipgate_dataset import clean_sipgate_dataset

In [ ]:
def js_divergence(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    m = 0.5 * (p + q)
    return 0.5 * (scipy.special.kl_div(p, m) + scipy.special.kl_div(q, m))

In [ ]:
generated_data = pd.read_csv("output/few_shot_simple_data.csv")
golden_data = clean_sipgate_dataset(read_sipgate_dataset())

In [ ]:
for metric in golden_data.columns:
    jsd = js_divergence(generated_data[metric], golden_data[metric])
    print(jsd)